# Lab 01 "Cân bằng phương trình hóa học"

**Môn**: MTH00051 "Toán ứng dụng và thống kê" @ 18CLC4

**SV**: 18127221, Bùi Văn Thiện

---

## Imports

In [1]:
import sympy as sp    # matrices and numbers
import IPython        # display inline Latex/MathJax

## Xử lý sơ bộ

In [2]:
# chem (nested list) -> chem (flattened list)
def FlattenMe(chem, mul=1):
    final_list = list()
    for each in chem:
        # Naive typecheck
        if type(each) is not tuple:
            raise ValueError('Invalid compound structure')
        # Explore and append new list
        if type(each[0]) is list:
            final_list.extend(FlattenMe(*each))
        # Append current list
        else:
            final_list.append((each[0], mul*each[1]))
    return final_list

In [3]:
# [chems] -> {elems} (set)
def FindMe(chems):
    all_elems = set()
    for each in chems:
        # Naive typecheck
        if type(each) is not list:
            raise ValueError('Invalid compound in list')
        # Make set from current chem
        each_elems = {x[0] for x in FlattenMe(each)}
        # Join with 'all' set
        all_elems = all_elems.union(each_elems)
    return all_elems

In [4]:
# chem -> {elem: count}
def CountMe(chem, elem_list):
    count_list = dict.fromkeys(elem_list, 0)
    for each in FlattenMe(chem):
        count_list[each[0]] += each[1]
    return count_list

## Cân bằng hệ số
(sử dụng các hàm ma trận của `SymPy`)

In [5]:
def BalanceMe(lhs, rhs):
    # Naive check
    if not (FindMe(lhs) == FindMe(rhs)):
        raise ValueError('Left-hand side and right-hand side mismatch')
    all_elems = FindMe(lhs)

    A = []
    for each in lhs:
        A.append([+c for c in CountMe(each, all_elems).values()])
    for each in rhs:
        A.append([-c for c in CountMe(each, all_elems).values()])
    
    almost_solution = [-x for x in sp.Matrix(A).T.rref()[0].col(-1)]
    if almost_solution[-1] == 0: almost_solution.pop()
    almost_solution.append(sp.Integer(1))
    
    # Alternative: span
    # sp.Matrix(A).T.nullspace()
    
    that_lcm = sp.lcm([x.q for x in almost_solution])
    solution = [x*that_lcm for x in almost_solution]
    return solution

## Biểu diễn thành LaTeX

In [6]:
# chem -> LaTeX string
def TeXifyMe(chem):
    final_string = ''
    for each in chem:
        if type(each) is not tuple:
            raise ValueError('Invalid compound structure')
        if type(each[0]) is list:
            final_string += (
                  ('(' if each[1] != 1 else '')
                + ('{}'.format(TeXifyMe(each[0])))
                + (')' if each[1] != 1 else '')
                + ('_{{{}}}'.format(each[1]) if each[1] != 1 else '')
                + ' '
            )
        else:
            final_string += (
                  ('\\text{{{}}}'.format(each[0]))
                + ('_{{{}}}'.format(each[1]) if each[1] != 1 else '')
                + ' '
            )
    return final_string.strip()

In [7]:
# equation -> LaTeX string
def TeXifyAllOfMe(lhs, rhs, solution):
    lhs_tex = ' + '.join([
        '{0}{1}'.format(
            coff if coff != 1 else ' ',
            TeXifyMe(chem)
        )
        for chem, coff in zip(lhs, solution[:len(lhs)])   
    ])
    rhs_tex = ' + '.join([
        '{0}{1}'.format(
            coff if coff != 1 else ' ',
            TeXifyMe(chem)
        )
        for chem, coff in zip(rhs, solution[len(lhs):])   
    ])
    final_solution_tex = '{0} \\to {1}'.format(lhs_tex, rhs_tex)
    return final_solution_tex

In [8]:
# Display everything (LaTeX rendering) in IPython
def DisplayMe(lhs, rhs, solution):
    IPython.display.display(
        IPython.display.Math(
            TeXifyAllOfMe(lhs, rhs, solution)
        )
    )

## VD

In [9]:
Fe = [('Fe', 1)]
Cl2 = [('Cl', 2)]
FeCl3 = [('Fe', 1), ('Cl', 3)]

# expected solution = [2, 3, 2]
lhs = [Fe, Cl2]
rhs = [FeCl3]
solution = BalanceMe(lhs, rhs)
DisplayMe(lhs, rhs, solution)

<IPython.core.display.Math object>

In [10]:
AlOH3 = [('Al', 1), ([('O', 1), ('H', 1)], 3)]
H2SO4 = [('H', 2), ([('S', 1), ('O', 4)], 1)]
Al2SO43 = [('Al', 2), ([('S', 1), ('O', 4)], 3)]
H2O = [('H', 2), ('O', 1)]

# expected solution = [2, 3, 1, 6]
lhs = [AlOH3, H2SO4]
rhs = [Al2SO43, H2O]
solution = BalanceMe(lhs, rhs)
DisplayMe(lhs, rhs, solution)

<IPython.core.display.Math object>

In [11]:
C6H5COOH = [('C', 6), ('H', 5), ([('C', 1), ('O', 1), ('O', 1), ('H', 1)], 1)]
O2 = [('O', 2)]
CO2 = [('C', 1), ('O', 2)]
H2O = [('H', 2), ('O', 1)]

# expected solution = [2, 15, 14, 6]
lhs = [C6H5COOH, O2]
rhs = [CO2, H2O]
solution = BalanceMe(lhs, rhs)
DisplayMe(lhs, rhs, solution)

<IPython.core.display.Math object>

In [12]:
KMnO4 = [('K', 1), ('Mn', 1), ('O', 4)]
HCl = [('H', 1), ('Cl', 1)]
KCl = [('K', 1), ('Cl', 1)]
MnCl2 = [('Mn', 1), ('Cl', 2)]
H2O = [('H', 2), ('O', 1)]
Cl2 = [('Cl', 2)]

# expected solution = [2, 16, 2, 2, 8, 5]
lhs = [KMnO4, HCl]
rhs = [KCl, MnCl2, H2O, Cl2]
solution = BalanceMe(lhs, rhs)
DisplayMe(lhs, rhs, solution)

<IPython.core.display.Math object>

In [13]:
K4FeCN6 = [('K', 4), ('Fe', 1), ([('C', 1), ('N', 1)], 6)]
H2SO4 = [('H', 2), ([('S', 1), ('O', 4)], 1)]
H2O = [('H', 2), ('O', 1)]
K2SO4 = [('K', 2), ([('S', 1), ('O', 4)], 1)]
FeSO4 = [('Fe', 1), ([('S', 1), ('O', 4)], 1)]
NH42SO4 = [([('N', 1), ('H', 4)], 2), ([('S', 1), ('O', 4)], 1)]
CO = [('C', 1), ('O', 1)]

# expected solution = [1, 6, 6, 2, 1, 3, 6]
lhs = [K4FeCN6, H2SO4, H2O]
rhs = [K2SO4, FeSO4, NH42SO4, CO]
solution = BalanceMe(lhs, rhs)
DisplayMe(lhs, rhs, solution)

<IPython.core.display.Math object>